In [171]:
import pandas as pd
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()
df = pd.read_csv('AWD-J_EX.txt',sep="\t")

df.sample(5)

,Word,Score,Deviation,PoS
212455,824年,2.20,49.99,名詞
363019,ゅうなごん,2.32,43.40,名詞
235808,石狩国,2.22,49.12,名詞
382593,LAメタル,2.34,41.81,名詞
230067,有効ポイント制,2.21,49.33,名詞


In [172]:
import math


df["integer_part"] = df["Score"].apply(lambda x: math.modf(x)[1])
df.sample(5)
df = df.loc[df["integer_part"] != 4.0]

In [173]:
unique_integer_paths = df["integer_part"].unique()
samples_by_integer_part = []

for score in unique_integer_paths:
    samples_by_integer_part.append(df.loc[df["integer_part"] == score].sample(10))

samples_by_integer_part

[        Word  Score  Deviation PoS  integer_part
 34397   事業計画   1.99      62.32  名詞           1.0
 17442  クレオール   1.89      67.93  名詞           1.0
 118    自動車部品   1.35      99.54  名詞           1.0
 35451  BLEND   1.99      62.09  名詞           1.0
 27437   粉砕骨折   1.96      64.11  名詞           1.0
 524      ハワイ   1.47      92.59  名詞           1.0
 18859    マイク   1.90      67.28  名詞           1.0
 20991     鳴物   1.92      66.35  名詞           1.0
 11602  ワイヤレス   1.83      71.41  名詞           1.0
 24129  路上ライブ   1.94      65.16  名詞           1.0,
                  Word  Score  Deviation PoS  integer_part
 297365    ジェームズ・ジョンソン   2.26      46.76  名詞           2.0
 264774           カーシャ   2.24      48.05  名詞           2.0
 397942           総裁任期   2.37      39.99  名詞           2.0
 317244     Gibberella   2.27      45.89  名詞           2.0
 380271           Liao   2.34      42.03  名詞           2.0
 297615           ラクタム   2.26      46.75  名詞           2.0
 304347         Fricke   2.26      4

In [196]:
def df_to_prompt_example(df):
    str = ""
    for row in df.iterrows():
        str += f"{row[1][0]}: {row[1][1]}\n"
    return str


In [197]:
prompt_example = ""
for df in samples_by_integer_part:
    prompt_example += df_to_prompt_example(df)
prompt = f"""
あなたは言葉の抽象度を評価するアシスタントです。
ユーザが提供したことの抽象度を判断してください。
scoreは1~4の間で評価してください。
以下を参考にしてください。
{prompt_example}

かならず以下のフォーマットに従ってください。
word: score
"""

prompt

(34397, Word             事業計画
Score            1.99
Deviation       62.32
PoS                名詞
integer_part      1.0
Name: 34397, dtype: object)
(17442, Word            クレオール
Score            1.89
Deviation       67.93
PoS                名詞
integer_part      1.0
Name: 17442, dtype: object)
(118, Word            自動車部品
Score            1.35
Deviation       99.54
PoS                名詞
integer_part      1.0
Name: 118, dtype: object)
(35451, Word            BLEND
Score            1.99
Deviation       62.09
PoS                名詞
integer_part      1.0
Name: 35451, dtype: object)
(27437, Word             粉砕骨折
Score            1.96
Deviation       64.11
PoS                名詞
integer_part      1.0
Name: 27437, dtype: object)
(524, Word              ハワイ
Score            1.47
Deviation       92.59
PoS                名詞
integer_part      1.0
Name: 524, dtype: object)
(18859, Word              マイク
Score             1.9
Deviation       67.28
PoS                名詞
integer_part      1.0
Name: 18859, d

/var/folders/4q/j50flbv96zz9cycb3pz1wq1h0000gn/T/ipykernel_5381/4031558319.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  str += f"{row[1][0]}: {row[1][1]}\n"


'\nあなたは言葉の抽象度を評価するアシスタントです。\nユーザが提供したことの抽象度を判断してください。\nscoreは1~4の間で評価してください。\n以下を参考にしてください。\n事業計画: 1.99\nクレオール: 1.89\n自動車部品: 1.35\nBLEND: 1.99\n粉砕骨折: 1.96\nハワイ: 1.47\nマイク: 1.9\n鳴物: 1.92\nワイヤレス: 1.83\n路上ライブ: 1.94\nジェームズ・ジョンソン: 2.26\nカーシャ: 2.24\n総裁任期: 2.37\nGibberella: 2.27\nLiao: 2.34\nラクタム: 2.26\nFricke: 2.26\n24kg: 2.08\nHorticultural: 2.17\nタルッフィ: 2.09\n乗じ: 3.68\n関: 3.78\nぶつ: 3.01\n明快: 3.06\n労: 3.12\n帰し: 3.15\n聴: 3.1\n尚早: 3.35\n漂っ: 3.2\n崔: 3.1\n\n\nかならず以下のフォーマットに従ってください。\nword: score\n'

In [176]:
client = OpenAI()
response = client.chat.completions.create(
    model="gpt-4-turbo-preview",
    messages=[
        {"role": "system", "content": prompt},
        {"role": "user", "content":"積極的なコミュニケーション,職位の壁を超えたコミュにケーション"}])

response.choices[0].message.content



'積極的なコミュニケーション: 2.60\n職位の壁を超えたコミュニケーション: 2.85'